In [2]:
import numpy as np

Keyword arguments: <br />
    txt             --  txt文本的文件地址或者是文本字符串 <br />
    stopwords       --  停用词数组 <br />
    window          --  一个句子中相邻的window个单词，两两之间认为有边 <br />
    m               --  想要获取的关键词个数 <br />

In [4]:
 def get_keywords(txt,stopwords,window=2,m=10):
    if txt.endswith('.txt'):
        words=txtToWords(txt,stopwords)
    else:
        words=docToWords(txt,stopwords)
    t_trans_mat = []
    result_words=[]
    result_prob=[]
    #从词查下标的字典
    word_index = {}
    #从下标查词的字典
    index_word = {}
    words_number = 0
    
    #以下将把words列表中所有词统计到VSM（向量空间模型）中，并提供词到下标，下标到词两个字典方便后续查找。
    for word_list in words:
        for word in word_list:
            if not word in word_index:
                word_index[word] = words_number
                index_word[words_number] = word
                words_number += 1

    #以下根据每个词出现的位置信息建立无向权重图
    graph =  np.zeros((words_number,words_number))
    #如果两个词出现在同一个句子中，符合窗口大小关系，则在它们之间建立一条边，并存入图中。
    for word_list in words:
        for w1, w2 in combine(word_list, window):
            if w1 in word_index and w2 in word_index:
                index1 = word_index[w1]
                index2 = word_index[w2]
                graph[index1][index2] += 1.0
                graph[index2][index1] += 1.0
    
    #在graph中获取前m个关键词词的下标，然后在index_word字典中查到词，存入result_words中返回
    w_prob=np.array(np.mean(graph,axis=0))
    w_prob_rank=np.argsort(-w_prob)
    for i in range(m):
        result_words.append(index_word[w_prob_rank[i+1]])
        result_prob.append(w_prob[w_prob_rank[i+1]])
    return result_words,result_prob